In [11]:
from bs4 import *
import requests as rp
import os
from msedge.selenium_tools import *
import time
from tqdm import tqdm
import pickle
import numpy as np
import psycopg2 as ps
import random

options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
#options.add_argument("disable-gpu")

In [2]:
pickle_in = open('zocdoc_remaining_doctor.pickle', 'rb')
cleaned_data_without_repetation = pickle.load(pickle_in)

In [3]:
def push_to_table(name, gender, npi, specialities, practice_name, eduandtrain, langspoken, lives, address, about, rating, review):
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute("insert into zocdoc_data values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (name, gender, npi, specialities, practice_name, eduandtrain, langspoken, lives, address, about, rating, review))
    conn.commit()
    conn.close()    

In [4]:
def get_page_data(string):
    out = {}
    data = []
    driver = Edge(options = options)
    driver.get(string)
    soup = BeautifulSoup(driver.page_source, 'html.parser')  
    name = soup.find('h1').text
    lives = soup.find_all('h2')[1].text
    specalist = soup.find('section',{'data-test':"Specialties-section"})
    practice = soup.find('section',{'data-test':"Practice-section"})
    education = soup.find('section',{"data-test":"Education-section"})
    language = soup.find('section',{'data-test':"Languages-section"})
    gender = soup.find('section',{'data-test':"Sex-section"})
    npi = soup.find('section',{'data-test':"NPI-section"})
    
    for i in [specalist, practice, education, language]:
        ind = i.find('h3').text
        data = []
        for j in i.find_all('li'):
            data.append(j.text)
        out[ind] = ",".join(data)

    out['npi'] = npi.find('p',{'itemprop':'identifier'}).text
    out['gender'] = gender.find('p').text
    out['name'] = name
    out['lives'] = lives
    out['address'] = string

    try:
        out['about'] = soup.find('span', {'class':'sc-1opoey3-2 itqzoH'}).text + soup.find('span', {'class':'sc-1opoey3-3 jSGFbe'}).text
    except:
        out['about'] = ''
    try:   
        b= soup.find_all('div', {'data-test':'location-card-address-container'})
        add = []
        for i in b:
            add.append(i.text)
            out['location'] = ";".join(add)
    except:
        out['location'] = ''
    try:
        out['rating'] = soup.find('div', {'data-test':'star-rating-score'}).text
    except:
        out['rating'] = ''
    try:
        out['review'] = soup.find('div', { 'data-test':"reviews-filter-panel-review-count"}).text.split(" ")[0]
    except:
        out['review'] = ''
        
    driver.close()
    return out

In [ ]:
for i in tqdm(cleaned_data_without_repetation[5039:]):
    try:
        out = get_page_data(i)
    except:
        continue
    push_to_table(out['name'], out['gender'], out['npi'], out['Specialties'], out['Practice names'], out['Education and training'], out['Languages spoken'], out['lives'], out['address'], out['about'], out['rating'], out['review'])

  2%|█▊                                                                     | 1499/60273 [4:41:08<222:40:30, 13.64s/it]

In [18]:
pickle_in = open('zocdoc.pickle', 'rb')
b = pickle.load(pickle_in)
for i in b:
    push_to_table(i['name'], i['gender'], i['npi'], i['Specialties'], i['Practice names'], i['Education and training'], i['Languages spoken'], i['lives'], i['address'], i['about'], i['rating'], i['review'])

# postman

In [1]:
import requests
import pprint
import pickle
import json

In [32]:
r = requests.get('https://server.mddocz.com/doctors/getDoc/603745067fcc6c59ed6631ee')

In [10]:
pickle_in = open('checkpoint2.pickle', 'rb')
previous_array2 = pickle.load(pickle_in)

In [22]:
dat = previous_array2[0]

In [30]:
'''
{
registration_number:100001
specialty:Dentist
basic:{}
password:DemoDoc@1234
city:New Delhi
state:Delhi
country:India
email:mpujari@gmail.com
firstName:M.R
lastName:Pujari
}
'''
payload = {'firstName':'Dr.',
          'lastName':dat['name'],
          'email':''.join(dat['name'].split(" "))+'@gmail.com',
          'country':'India',
          'state':dat['lives'].split(",")[-1],
          'city':dat['lives'].split(",")[-1],
          'password':'DemoDoc@1234',
          'basic':{},
          'specialty':dat['specilization'],
          'registration_number':'10003'}
r = requests.post('https://server.mddocz.com/doctors/register', json = payload)

In [38]:
data = json_obj['data']

In [39]:
received_id = data['_id']

In [40]:
data.keys()

dict_keys(['document', 'address', 'taxonomies', 'practiceLocation', 'diseases', 'description', 'steps', 'is_in_network', 'is_superDoc', 'location_ids', 'isActive', 'isApproved', 'availability', 'appointments', 'picture', 'profileStatus', 'question', 'appointmentsString', 'onBoarding', 'referrals', 'latestAppointment', 'payment', 'isAssistant', 'rights', 'assistants', 'charity', 'clinic', 'block', 'deviceToken', 'online', 'socket', 'conversations', 'WaitingRoomSocket', 'verify', '_id', 'npi', 'basic', 'email', 'password', 'specialty', 'customerProfile', 'city', 'state', 'country', 'firstName', 'referralId', 'createdOn', 'education', 'PatientsWaiting', '__v'])

In [33]:
r.json()['data']

{'basic': {'organization_name': 'not found',
  'organizational_subpart': 'not found',
  'enumeration_date': 'not found',
  'last_updated': 'not found',
  'status': 'not found',
  'telephone_number': 'not found',
  'title_or_position': 'not found',
  'name_prefix': 'not found',
  'name_suffix': 'not found',
  'sole_proprietor': 'not found',
  'gender': 'not found',
  'name': 'Dr. Dr. Saumya Shetty'},
 'document': {'idProof': False,
  'registrationProof': False,
  'establishment': False},
 'address': [],
 'taxonomies': [],
 'practiceLocation': [],
 'diseases': [],
 'description': 'NA',
 'steps': [0, 0, 0, 0],
 'is_in_network': True,
 'is_superDoc': False,
 'location_ids': [],
 'isActive': True,
 'isApproved': False,
 'availability': [],
 'appointments': [{'cancelledBy': {'cancelledByPatient': False,
    'cancelledByDoctor': False},
   'bookedFor': '2021-02-25T10:30:00.000Z',
   'paid': False,
   'booked': False,
   'approved': False,
   'quiz': [],
   '_id': '603745097fcc6c59ed6631f1',
 

In [ ]:
r = requests.post('https://server.mddocz.com/doctors/profile/update', json = payload)

In [35]:
dat

{'name': 'Dr. Saumya Shetty',
 'qualification': 'MBBS, MD - Dermatology , Venereology & Leprosy',
 'lives': '305, The Central Building, Shell Colony Road, Landmark: Near Old Jenny Photo Studio, Mumbai',
 'specilization': 'Dermatologist,Cosmetologist,Special Interest in Trichology',
 'education': 'MBBS - Bharati Vidyapeeth University, Pune, 2011,MD - Dermatology , Venereology & Leprosy - Krishna Institute of Medical Sciences, Karad, 2014'}

In [73]:
'''payload = {'firstName':'Dr.',
          'lastName':dat['name'],
          'email':''.join(dat['name'].split(" "))+'@gmail.com',
          'country':'India',
          'state':dat['lives'].split(",")[-1],
          'city':dat['lives'].split(",")[-1],
          'password':'DemoDoc@1234',
          'basic':{},
          'specialty':dat['specilization'],
          'registration_number':'10003'}'''
payload = {
    'id': '603745067fcc6c59ed6631ee',
    'education':entry
}
r = requests.post('https://server.mddocz.com/doctors/profile/update', json = payload)

In [47]:
r = requests.get('https://server.mddocz.com/doctors/getDoc/603745067fcc6c59ed6631ee')

In [50]:
r.json()['data']

{'basic': {'organization_name': 'not found',
  'organizational_subpart': 'not found',
  'enumeration_date': 'not found',
  'last_updated': 'not found',
  'status': 'not found',
  'telephone_number': 'not found',
  'title_or_position': 'not found',
  'name_prefix': 'not found',
  'name_suffix': 'not found',
  'sole_proprietor': 'not found',
  'gender': 'not found',
  'name': 'Dr. Dr. Saumya Shetty'},
 'document': {'idProof': False,
  'registrationProof': False,
  'establishment': False},
 'address': [],
 'taxonomies': [],
 'practiceLocation': [],
 'diseases': [],
 'description': 'NA',
 'steps': [0, 0, 0, 0],
 'is_in_network': True,
 'is_superDoc': False,
 'location_ids': [],
 'isActive': True,
 'isApproved': False,
 'availability': [],
 'appointments': [{'cancelledBy': {'cancelledByPatient': False,
    'cancelledByDoctor': False},
   'bookedFor': '2021-02-25T10:30:00.000Z',
   'paid': False,
   'booked': False,
   'approved': False,
   'quiz': [],
   '_id': '603745097fcc6c59ed6631f1',
 

In [59]:
entry = dat['education']

In [61]:
def spliter(data):
    i=0
    j=0
    arr = []
    print(len(data))
    while(j+1!=len(data)):
        try:
            if(data[j] == ',' and data[j+1] != ' ' ):
                m = data[i:j]
                arr.append(m)
                
                i = j+1
                j = j+2
            else:
                j+=1
        finally:
            j+=1
    arr.append(data[i:j+1])
    return arr

In [63]:
prep = spliter(entry)

142


In [71]:
entry = []
for i in prep:
    x = i.split(', ')
    entry.append(
        {
            'degree':x[0].split(' - ')[0],
            'university':x[0].split(' - ')[1],
            'year':x[-1]
        }
    )    

In [72]:
entry

[{'degree': 'MBBS',
  'university': 'Bharati Vidyapeeth University',
  'year': '2011'},
 {'degree': 'MD', 'university': 'Dermatology ', 'year': '2014'}]

In [4]:
#getting data from database
def retrive_data():
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute("select * from zocdoc_data")
    data = cur.fetchall()
    return data

In [5]:
data = retrive_data()

In [ ]:
for i in tqdm(data[(4000 + 30):]):
    send_data(i)

  0%|                                                                               | 1/1009 [00:23<6:36:32, 23.60s/it]

something wrong <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>



  0%|▏                                                                              | 2/1009 [00:23<2:46:16,  9.91s/it]

something wrong <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>



  0%|▏                                                                              | 3/1009 [00:24<1:32:26,  5.51s/it]

...

something wrong <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>



  8%|██████▌                                                                         | 83/1009 [00:48<04:35,  3.36it/s]

something wrong <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>



  8%|██████▋                                                                         | 84/1009 [00:49<04:40,  3.30it/s]

something wrong <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>



In [18]:
wrong = []
something_wrong = []
def send_data(out):
    #inserting name
    try:
        name = out[0].split(',')[0].split(" ")
        middle_name = ""
        first_name = name[1]
        last_name = ' '.join(name[2:])


        #inserting gender
        gender = out[1]

        #inserting npi
        npi = out[2]

        #inserting speciality
        speciality = out[3].split(",")[0]

        #inserting about
        about = out[9]

        #inserting email
        email = out[8].split('/')[-1]+str(random.randint(1,1000))+"@gmail.com"

        #inserting address
        country = 'United States'
        city = out[7].split(',')[0]
        state = out[7].split(',')[1][1:]

        #inserting password
        password = 'DemoDoc@1234'
        verify = 'True'

        #inserting fees
        fee_list = [200, 500, 1000, 700, 600, 1500, 2000]
        fee = random.choice(fee_list)

        #education

        degree = out[0].split(',')[1:]
        university = out[5].split('- ')[1].split(',')[0]
        edu = []
        for i in degree:
             edu.append(
                 {
                     'degree':i[1:],
                     'university':university,
                     'year':''
                 }
             )


        #register doctors
        payload = {'firstName':first_name,
                  'lastName':last_name,
                  'email':email,
                  'country':country,
                  'state':state,
                  'city':city,
                  'password':password,
                  'basic':{},
                  'specialty':speciality,
                  'registration_number':npi}
        try:
            r = rp.post('https://server.mddocz.com/doctors/register', json = payload)


            get_id = r.json()['data']['_id'] #id of created
            datas[email] = get_id

            if(len(datas.keys()) % 100 == 0):
                pickle_out = open('inserted_postman5.pickle','wb')#change for another checkpoint
                pickle.dump(datas, pickle_out)
                pickle_out.close()


            #update_doctors
            payload = {
                'gender':gender,
                'id':get_id,
                'bio':about,
                'education': edu,
                'fee':fee,
                'verify':'true',
            }
            r = rp.post('https://server.mddocz.com/doctors/profile/update', json = payload)

        except:
            something_wrong.append(out)
            print('something wrong', r.text)
    except:
        print('error')
        wrong.append(out)

In [16]:
datas

NameError: name 'datas' is not defined

In [15]:
pickle_out = open('inserted_postman5.pickle','wb')#change for another checkpoint
pickle.dump(datas, pickle_out)
pickle_out.close()

NameError: name 'datas' is not defined

In [22]:
pickle_check = open('inserted_postman5.pickle', 'rb')
s = pickle.load(pickle_check)

In [23]:
len(s)

2647

In [ ]:
'''
{'aaron-diehl-dds-59885@gmail.com': '603f8f089b38c41335aec263',
 'aaron-elkowitz-dmd-168924@gmail.com': '603f8f0e9b38c41335aec279',
 'aaron-johnson-dds-37463@gmail.com': '603f8f139b38c41335aec28f',
 'aaron-soeprono-dds-3151@gmail.com': '603f8f189b38c41335aec2a5',
 'danielle-kerger-dds-83188@gmail.com': '603f8f1e9b38c41335aec2bb',
 'aaron-lee-dds-39094@gmail.com': '603f8f239b38c41335aec2d1',
 'aaron-moskowitz-dds-133560@gmail.com': '603f8f289b38c41335aec2e7',
 'aaron-nicholas-dds-89983@gmail.com': '603f8f329b38c41335aec2ff'}
'''

In [6]:
len(data)

5039

In [17]:
datas ={}

In [9]:
out = data[30+3000]
name = out[0].split(',')[0].split(" ")
middle_name = ""
first_name = name[1]
last_name = ' '.join(name[2:])


#inserting gender
gender = out[1]

#inserting npi
npi = out[2]

#inserting speciality
speciality = out[3].split(",")[0]

#inserting about
about = out[9]

#inserting email
email = out[8].split('/')[-1]+"@gmail.com"

#inserting address
country = 'United States'
city = out[7].split(',')[0]
state = out[7].split(',')[1][1:]

#inserting password
password = 'DemoDoc@1234'
verify = 'True'

#inserting fees
fee_list = [200, 500, 1000, 700, 600, 1500, 2000]
fee = random.choice(fee_list)

#education

degree = out[0].split(',')[1:]
university = out[5].split('- ')[1].split(',')[0]
edu = []
for i in degree:
     edu.append(
         {
             'degree':i[1:],
             'university':university,
             'year':''
         }
     )


#register doctors
payload = {'firstName':first_name,
          'lastName':last_name,
          'email':email,
          'country':country,
          'state':state,
          'city':city,
          'password':password,
          'basic':{},
          'specialty':speciality,
          'registration_number':npi}
response_back = rp.post('https://server.mddocz.com/doctors/register', json = payload)



In [16]:
get_id = response_back.json()['data']['_id'] #id of created
datas[email] = get_id

if(len(datas.keys()) % 100 == 0):
    pickle_out = open('inserted_postman5.pickle','wb')#change for another checkpoint
    pickle.dump(datas, pickle_out)
    pickle_out.close()


#update_doctors
payload = {
    'gender':gender,
    'id':get_id,
    'bio':about,
    'education': edu,
    'fee':fee,
    'verify':'true',
}
r = rp.post('https://server.mddocz.com/doctors/profile/update', json = payload)

In [17]:
print(r.text)

{"status":true,"message":"Profile successfully updated","data":{"basic":{"organization_name":"not found","organizational_subpart":"not found","enumeration_date":"not found","last_updated":"not found","status":"not found","telephone_number":"not found","title_or_position":"not found","name_prefix":"not found","name_suffix":"not found","sole_proprietor":"not found","gender":"not found","name":"Edmond Massabni"},"document":{"idProof":false,"registrationProof":false,"establishment":false},"address":[],"taxonomies":[],"practiceLocation":[],"diseases":[],"description":"NA","steps":[0,0,0,0],"is_in_network":true,"is_superDoc":false,"location_ids":[],"isActive":true,"isApproved":false,"availability":[],"appointments":["6040a0ec4393163d34419226","6040a0ec4393163d34419227","6040a0ec4393163d34419228","6040a0ec4393163d34419229","6040a0ec4393163d3441922a","6040a0ec4393163d3441922b","6040a0ec4393163d3441922c","6040a0ec4393163d3441922d","6040a0ec4393163d3441922e","6040a0ec4393163d3441922f","6040a0ec4

In [13]:
r = rp.get('https://server.mddocz.com/doctors/getDoc/603f92839b38c41335aec381')

In [15]:
datas = {}

In [14]:
r.text

'{"status":true,"message":"Fetched","data":{"basic":{"organization_name":"not found","organizational_subpart":"not found","enumeration_date":"not found","last_updated":"not found","status":"not found","telephone_number":"not found","title_or_position":"not found","name_prefix":"not found","name_suffix":"not found","sole_proprietor":"not found","gender":"not found","name":"Abbe Kellner-Kutno"},"document":{"idProof":false,"registrationProof":false,"establishment":false},"address":[],"taxonomies":[],"practiceLocation":[],"diseases":[],"description":"NA","steps":[0,0,0,0],"is_in_network":true,"is_superDoc":false,"location_ids":[],"isActive":true,"isApproved":false,"availability":[],"appointments":[{"cancelledBy":{"cancelledByPatient":false,"cancelledByDoctor":false},"bookedFor":"2021-03-03T10:30:00.000Z","bookedOn":"2021-03-03T11:19:29.513Z","paid":false,"booked":false,"approved":false,"quiz":[],"_id":"603f92879b38c41335aec385","doctor":"603f92839b38c41335aec381","createdAt":"2021-03-03T13

In [12]:
driver = Edge(options = options)
for i in range(1,3):
    driver.get('https://paper.erudition.co.in/makaut/btech-in-computer-science-and-engineering/5/software-engineering/chapter/coding-documentation-and-testing/group-1/page-'+str(i))
    soup = BeautifulSoup(driver.page_source, 'html.parser')  
    for i in soup.find_all('p'):
        print(i.text)

Multiple Choice Type QuestionsChoose the correct alternatives for the following.
1. A state diagram (a) is not related to object modeling technology (b) does not have a node (c) Is a graph having the arcs and transitions (d) none of these
2. The scope of a design must be (a) bounded (b) unbounded (c) not relevant for design (d) none of these
3. A decision table is  (a) a truth table (b) a table that facilitates taking decisions (c) a table listing conditions and actions to be taken based on the testing of conditions (d) a table in a decision support system
4. Quality assurance CASE tools are (a) OODMS (c) Source Code auditors (b) data bridges (d) none of these
5. Documentation is prepared at the time of (a) commencing the study (b) every state (c) completion of the design (d) completion of implementation
6. A system that does not interact with external environment is called (a) closed system (b) logical system (c) open system (d) hierarchical system
7. Which of the following is not str

In [13]:
driver.close()

In [2]:
import urllib.request
import time
import os
from bs4 import *
import requests as rp
import os
from msedge.selenium_tools import *
import time
from tqdm import tqdm
import pickle
import numpy as np
import psycopg2 as ps
import random
from selenium.webdriver.common.by import *
import selenium
import ast
import concurrent.futures
import threading
import tensorflow as tf
import json


options = EdgeOptions()
options.use_chromium = True
#options.add_argument("headless")
options.add_argument("disable-gpu")

#pickle_in = open('extraurls.pickle', 'rb')
#zips = pickle.load(pickle_in)
#pickle_in.close()

def retrive_data():
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute('select distinct url from zoc_doc_urls')
    data = cur.fetchall()
    return data
def push_to_table_zocdocdata(fname, lname, gender, npi, speciality, email,country,state,city,verify,password,degree,university,year,fee,bio,experience, url, location, languagespoken, review, rating):
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute('insert into "zoc_doc_data" values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', (fname, lname, npi, speciality, email,country,state,city,verify,password,degree,university,year,fee,bio,experience, url, gender, location, languagespoken, review, rating))
    conn.commit()
    conn.close()
def save_image(url, file_name, file_path = 'E:/Youtube game/works/zocdoc/'):
    try:
        full_path = file_path+file_name+'.png'
        if os.path.isfile(full_path) == True:
            cpy.append(full_path)
        urllib.request.urlretrieve(url, full_path)
    except:
        print('error', file_name, url)
        pass

'''
ret_data = retrive_data()
rest_urls = []
count = 0
for i in ret_data:
    if (i[0].split("/"))[3][:6] == 'search':
        rest_urls.append(i[0])
'''

#database info
doctors_urls = []
ret_data = retrive_data()
count = 0
for i in ret_data:
    if (i[0].split("/"))[3][:6] == 'search':
        count+=1
    if (i[0].split("/"))[3][:6] != 'search':
        doctors_urls.append(i[0])
print('urls:',count)
print('doctors:', len(ret_data) - count)
print(len(doctors_urls))

#pushing urls to database
'''
def push_to_table(url, zip):
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute("insert into " +'zoc_doc_urls ' +"values ('{0}, {1}')".format(url, zip))
    conn.commit()
    conn.close()
'''
#url collection
'''
threadLimiter = threading.BoundedSemaphore(20)
def url_collector(url, zip):
    driver = Edge('E:/Django/msedgedriver.exe',options = options)
    driver.set_window_size(10, 10)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', {'data-test':'doctor-card-info-name'})
    for i in links:
        i = 'https://www.zocdoc.com'+str(i['href'])
        push_to_table(i, zip)
    driver.close()

    threadLimiter.release()

for zip in tqdm(rest_urls[150:200]):
    for page in tqdm(range(0,11)):
        name = str(zip)+'&offset='+str(page)
        print(name)
        t= threading.Thread(target = url_collector, args = [name, ''])
        threadLimiter.acquire()
        try:
            t.start()
        except:
            continue

print("================================================================================================ \n DONE \n ================================================================================================")
'''

def doc_data(url):
    
    file = 'E:/Youtube game/works/zocdoc/'

    driver = Edge('E:/Django/msedgedriver.exe',options = options)
    driver.set_window_size(10, 10)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        image = soup.find('img', {'data-test':'show-gallery'})['src']
    except:
        image_not_found.append(url)
        driver.close()
        threadLimiter.release()
        return 0

    name = soup.find('h1', {'class':'sc-9yjfx1-7 iqIZAV'}).text
    name_lis = name.split(",")[0].split()
    c= 1
    if(name_lis[0] != "Dr."):
        c-=1
    lname = ''
    fname = name_lis[c]
    try:
        lname = ' '.join(name_lis[c+1:])
    except:
        pass

    try:
        gender = (soup.find('section', {'data-test':'Sex-section'})).find('p').text
    except:
        return 0

    try:
        npi = (soup.find('section', {'data-test':'NPI-section'})).find('p').text
    except:
        driver.close()
        threadLimiter.release()
        return 0
        

    try:
        specilities = (soup.find('section', {'data-test':'Specialties-section'})).find_all('li')
        for i in range(len(specilities)):
            specilities[i] = (specilities[i].text)
        specilities = (";".join(specilities))
    except:
        specilities = ''
    

    email = str(npi)+'_' + fname + lname + '@gmail.com'
    
    try:
        loc = soup.find('div', {'data-test':'location-card-address-container'}).text
        country = 'United States'
        state = loc.split(', ')[0]
        city = loc.split(', ')[1]
    except:
        country = 'United States'
        state = ''
        city = ''

    verify = 'True'
    password = 'DemoDoc@1234'

    try:
        bio = soup.find('span', {'data-test':'preview-span'}).text + soup.find('span', {'class':'sc-1opoey3-3 jSGFbe'}).text
    except:
        bio = ''

    experience = 5

    education = []
    try:
        ed = soup.find('ul',{'data-test':'education-list'}).find_all('li')
        for i in ed:
            education.append({
            'year': 1990,
            'degree': '',
            'university': i.text
        })
    except:
        driver.close()
        threadLimiter.release()
        return 0
    
    language = []
    try:
        language_temp = soup.find('section', {'data-test':'Languages-section'})
        for i in language_temp.find_all('li'):
            language.append(i.text)
    except:
        pass
    language = ", ".join(language)
    
    try:
        rating = (soup.find('div', {'class':'qfkn0u-2 hgiJxr'})).text
    except:
        rating = '0'
    

    review = []
    for i in soup.find_all('div', {'itemprop':'reviewBody'}):
        review.append(i.text)
    review = str(review)

    location = []
    for i in soup.find_all('div', {'class':'g58yd9-6 hAgxrm'}):
        location.append(i.text)
    location = str(location)

    fee = 100
    file += npi+'.png' 
    save_image('http:'+image, email)
    push_to_table_zocdocdata(fname, lname, gender, npi, specilities, email, country, state, city, verify, password, str(education), str(education), str(file), fee, bio, experience, url, location, language, review, str(rating))

    driver.close()
    threadLimiter.release()



threadLimiter = threading.BoundedSemaphore(40)

for i in tqdm(doctors_urls[10000:]):
    t= threading.Thread(target = doc_data, args = [i])
    threadLimiter.acquire()
    t.start()

  7%|█████▏                                                                          | 28/429 [00:00<00:01, 272.38it/s]

urls: 7552
doctors: 10429
10429
error 1720420979_JustinLai@gmail.com http://d3wnzga3fpd9a.cloudfront.net/photos/Dr-Justin-Lai-DDS-214944-circle_large__v2__.png


  7%|█████▏                                                                          | 28/429 [00:20<00:01, 272.38it/s]

error 1710113394_KelleyBrown@gmail.com http://d1k13df5m14swc.cloudfront.net/photos/Kelley-Brown-LPC-MA-279535-circle_large__v1__.png
error 1861891566_ChristinaSchrack@gmail.com http://d3wnzga3fpd9a.cloudfront.net/photos/Christina-Schrack-NP-300215-circle_large__v1__.png
error 1023465424_StephanieLee@gmail.com http://d1k13df5m14swc.cloudfront.net/photos/Dr-Stephanie-Lee-DO-291665-circle_large__v1__.png
error 1578820379_NeetaMehta@gmail.com http://d1k13df5m14swc.cloudfront.net/photos/Dr-Neeta-Mehta-MD-303696-circle_large__v1__.png
error 1609938885_NaheedEsmail@gmail.com http://d3wnzga3fpd9a.cloudfront.net/photos/Dr-Naheed-Esmail-DO-305175-circle_large__v1__.png


 10%|███████▋                                                                         | 41/429 [00:40<08:04,  1.25s/it]

...

 98%|██████████████████████████████████████████████████████████████████████████████▋ | 422/429 [09:49<00:08,  1.16s/it]

error 1982045498_KatherineCarroll@gmail.com http://d2uur722ua7fvv.cloudfront.net/photos/Dr-Katherine-Carroll-DO-308252-circle_large__v1__.png


Exception in thread Thread-425:
Traceback (most recent call last):
  File "<ipython-input-2-a2fd38cd9828>", line 140, in doc_data
    image = soup.find('img', {'data-test':'show-gallery'})['src']
TypeError: 'NoneType' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\software\anaconda\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\software\anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-a2fd38cd9828>", line 142, in doc_data
    image_not_found.append(url)
NameError: name 'image_not_found' is not defined

100%|████████████████████████████████████████████████████████████████████████████████| 429/429 [09:56<00:00,  1.39s/it]


error 1699336560_RobertGeis@gmail.com http://d1k13df5m14swc.cloudfront.net/photos/Dr-Robert-Geis-DDS-310257-circle_large__v1__.png
error 1396158069_LukeKim@gmail.com http://d2uur722ua7fvv.cloudfront.net/photos/Dr-Luke-Kim-MD-289973-zoom.jpg
error 1508880279_JaishreeKumari@gmail.com http://d1k13df5m14swc.cloudfront.net/photos/Dr-Jaishree-Kumari-MD-186459-circle_large__v1__.png


Exception in thread Thread-434:
Traceback (most recent call last):
  File "<ipython-input-2-a2fd38cd9828>", line 140, in doc_data
    image = soup.find('img', {'data-test':'show-gallery'})['src']
TypeError: 'NoneType' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\software\anaconda\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\software\anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-a2fd38cd9828>", line 142, in doc_data
    image_not_found.append(url)
NameError: name 'image_not_found' is not defined



In [1]:
image_not_found = []

In [2]:
cpy = []

In [4]:
len(cpy)

0

In [3]:
import psycopg2 as ps
def retrive_data():
    conn = ps.connect(
        host = '127.0.0.1',
        database = 'zocdoc',
        user = 'postgres',
        password = 'ss123546',
    )
    cur = conn.cursor()
    cur.execute('select distinct npi from zoc_doc_data')
    data = cur.fetchall()
    return data
print(len(retrive_data()))

5316
error 1356500011_HylandCronin@gmail.com http://d1k13df5m14swc.cloudfront.net/photos/Dr-Hyland-Cronin-MD-FAAD-192217-circle_large__v1__.png
